# Load packages

# Load data

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split
from sklearn.metrics import brier_score_loss

import xgboost as xgb
from xgboost import XGBClassifier

In [7]:
#all_games_df = pd.read_csv('Data/~Created data/all_games_df.csv')

path = "C:/Users/anovitt/AppData/Local/Programs/Python/Python36/Scripts/kaggle_mania_2020_men/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/"

In [8]:
#data_matrix_df = pd.read_csv('data_matrix.csv')

data_matrix_df = pd.read_csv(path + 'data_matrix.csv')
sub_matrix_df = pd.read_csv(path + 'sub_matrix.csv')

In [9]:
data_matrix_df.head(3)

,Season,DayNum,T1,T1_Points,T2,T2_Points,Location,ResultDiff,X1_WinRatio14d,X1_Points_mean,...,X2_OppFgaMin,X1_Seed,X2_Seed,SeedDiff,X1_quality_march,X2_quality_march,T1_AP,T2_AP,T1_25Wins,T2_25Wins
0,2003,10,1104,68,1328,62,N,6,0.333333,69.285714,...,43,10,1,9,0.958527,1.044071,200.0,3.0,3,24
1,2003,10,1272,70,1393,63,N,7,0.750000,74.517241,...,52,7,3,4,1.049830,1.025533,19.0,13.0,23,24
2,2003,11,1266,73,1437,61,N,12,0.500000,78.392857,...,36,3,0,0,1.073925,0.000000,9.0,200.0,23,1


In [10]:
XX_train = data_matrix_df.drop(['Season','DayNum','T1','T1_Points','T2','T2_Points','Location','ResultDiff','T1_AP','T2_AP'],axis =1)

In [11]:
XX_train.head(3)

,X1_WinRatio14d,X1_Points_mean,X1_Points_sd,X1_Points_median,X1_fgm_mean,X1_fga_mean,X1_fgp,X1_fgm3_mean,X1_fga3_mean,X1_fg3p,...,X2_OppFgm3Mean,X2_OppFga3Mean,X2_OppFgaMin,X1_Seed,X2_Seed,SeedDiff,X1_quality_march,X2_quality_march,T1_25Wins,T2_25Wins
0,0.333333,69.285714,11.375273,69.0,24.035714,57.178571,0.420362,6.357143,19.857143,0.320144,...,4.433333,13.566667,43,10,1,9,0.958527,1.044071,3,24
1,0.750000,74.517241,8.724435,73.0,26.275862,60.000000,0.437931,7.000000,20.068966,0.348797,...,6.931034,22.586207,52,7,3,4,1.049830,1.025533,23,24
2,0.500000,78.392857,12.154375,75.5,27.214286,56.250000,0.483810,5.785714,15.250000,0.379391,...,7.266667,19.366667,36,3,0,0,1.073925,0.000000,23,1


In [12]:
sub_matrix_df.head(3)

,Unnamed: 0,ID,Pred,Season,T1,T2,X1_WinRatio14d,X1_Points_mean,X1_Points_sd,X1_Points_median,...,X2_OppFgaMin,X1_Seed,X2_Seed,SeedDiff,X1_quality_march,X2_quality_march,T1_AP,T2_AP,T1_25Wins,T2_25Wins
0,1,2015_1107_1112,0.5,2015,1107,1112,1.0,65.5,9.235276,64.0,...,39,14,2,12,0.74543,2.928111,200,200,0,0
1,2,2015_1107_1116,0.5,2015,1107,1116,1.0,65.5,9.235276,64.0,...,45,14,5,9,0.74543,1.189625,200,21,0,26
2,3,2015_1107_1124,0.5,2015,1107,1124,1.0,65.5,9.235276,64.0,...,39,14,3,11,0.74543,1.702049,200,200,0,0


In [13]:
X_test = sub_matrix_df.drop(['Season','ID','T1','T2','Pred','Unnamed: 0','T1_AP','T2_AP'],axis =1)

In [14]:
X_test.head(3)

,X1_WinRatio14d,X1_Points_mean,X1_Points_sd,X1_Points_median,X1_fgm_mean,X1_fga_mean,X1_fgp,X1_fgm3_mean,X1_fga3_mean,X1_fg3p,...,X2_OppFgm3Mean,X2_OppFga3Mean,X2_OppFgaMin,X1_Seed,X2_Seed,SeedDiff,X1_quality_march,X2_quality_march,T1_25Wins,T2_25Wins
0,1.0,65.5,9.235276,64.0,22.0625,50.15625,0.439875,6.15625,17.09375,0.360146,...,5.323529,16.205882,39,14,2,12,0.74543,2.928111,0,0
1,1.0,65.5,9.235276,64.0,22.0625,50.15625,0.439875,6.15625,17.09375,0.360146,...,6.235294,17.647059,45,14,5,9,0.74543,1.189625,0,26
2,1.0,65.5,9.235276,64.0,22.0625,50.15625,0.439875,6.15625,17.09375,0.360146,...,5.718750,19.250000,39,14,3,11,0.74543,1.702049,0,0


In [15]:

data_matrix_df['Win'] = 0

data_matrix_df.loc[data_matrix_df['T1_Points'] > data_matrix_df['T2_Points']] = 1


In [16]:
yy_train = data_matrix_df['Win']

In [17]:
X_train, X_val, y_train, y_val = train_test_split(XX_train, yy_train, test_size=0.25, random_state=42)


#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Note: test is 2019 predictions but our "test" holdout set is referred to as "valid" 

In [18]:
X_test = X_test.astype("float64")
XX_train = XX_train.astype("float64") # Full data set, use for round 2
X_train = X_train.astype("float64")
X_val = X_val.astype("float64")
#y_train = y_train.astype("float64")
#y_val = y_val.astype("float64")

# Scoring rules and two benchmarks

In [19]:
def LogLoss(predictions, realizations):
    predictions_use = predictions.clip(0)
    realizations_use = realizations.clip(0)
    LogLoss = -np.mean( (realizations_use * np.log(predictions_use)) + 
                        (1 - realizations_use) * np.log(1 - predictions_use) )
    return LogLoss

# Fit a neural network (with and without line)

Normalize data (using z-scores) before neural network

In [20]:
scaler = StandardScaler()
scaler.fit(X_train)  # Fit only to the training data
scaler.fit(X_val)
scaler.fit(X_test)
scaler.fit(XX_train)
scaled_X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
scaled_XX_train = pd.DataFrame(scaler.transform(XX_train), index=XX_train.index, columns=X_train.columns) # full data set
scaled_X_valid = pd.DataFrame(scaler.transform(X_val), index=X_val.index, columns=X_val.columns)
scaled_X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

With line (note: we won't have line in the rounds after the first, but we could use this for the first round only like Lopez and Matthews did) 

In [22]:
#Note: I tried logistic activation and different combinations of hidden layers/nodes
#Hyperparameters below minimized the log loss in the holdout set
#I also submited a prediction with 10 nodes in the first layer, but this is the submission that placed 4th (w/ 8 in 1st)
#nn = MLPClassifier(activation='relu', hidden_layer_sizes=(8,5,3),random_state=201, max_iter=1000)
nn = MLPClassifier(activation='relu', hidden_layer_sizes=(8,8,8,5,3),random_state=201, max_iter=1000)
nn.fit(scaled_X_train,y_train)
nn_pred = pd.DataFrame(nn.predict_proba(scaled_X_valid))[1]

In [26]:
# Full data set, use for round 2
nn = MLPClassifier(activation='relu', hidden_layer_sizes=(8,8,8,5,3),random_state=201, max_iter=1000)
nn.fit(scaled_XX_train,yy_train)
nn_pred = pd.DataFrame(nn.predict_proba(scaled_X_test))[1]

In [23]:
LogLoss(nn_pred, y_val)

0.8978296182159777

In [27]:
#try second holdout (does worse, but still better than baseline of 54)
#nn = MLPClassifier(activation='relu', hidden_layer_sizes=(8,8,8,5,3),random_state=201, max_iter=1000)
#nn.fit(scaled_XX_train,yy_train)
nn_pred_test = pd.DataFrame(nn.predict_proba(scaled_X_test))[1]

In [28]:
# write out prob csv file

nn_pred_test.to_csv(path + 'submitProb.csv', index=False)

***1D CNN Netwok Try***

In [5]:
# CNN 1D Network

#import keras

from tensorflow.keras import backend
#import keras

#from matplotlib import pyplot
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Flatten
#from keras.layers import Dropout
#from keras.layers.convolutional import Conv1D
#from keras.layers.convolutional import MaxPooling1D
#from keras.utils import to_categorical

In [49]:
scaled_X_train.shape

(90777, 119)

In [57]:
def evaluate_model(X_train, y_train, X_val, y_val):
    
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps = 1 #X_train.shape[2]
    n_features = X_train.shape[2]
    n_outputs = 1 #y_train.shape[1]
    
    #n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], Y_train.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=0)
    return accuracy

In [58]:
# run an experiment
def run_experiment(repeats=10):
    # load data
	
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(scaled_X_train,y_train, scaled_X_valid, y_val)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [59]:
run_experiment()

IndexError: tuple index out of range

In [34]:
print('hello')

hello


Without line

# Create test predictions

In [35]:
#Note: I'm predicting home (lower seed) win probability. Need to convert to be consistent with output file (lower team ID)
second_rd_submission = pd.merge(test_combos_df, second_rd_submission_all, left_index=True, right_index=True)

In [36]:
second_rd_submission.loc[second_rd_submission['HTeamID']<second_rd_submission['RTeamID'], 'pred'] = second_rd_submission[1]
second_rd_submission.loc[second_rd_submission['HTeamID']>second_rd_submission['RTeamID'], 'pred'] = second_rd_submission[0]

In [37]:
second_rd_submission.to_csv('Data/~Created data/second_rd_submission_all.csv', index=False)

In [38]:
second_rd_submission = second_rd_submission[['ID','pred']]

In [39]:
second_rd_submission.head()

,ID,pred
0,2019_1101_1113,0.190039
1,2019_1101_1120,0.078436
2,2019_1101_1124,0.122828
3,2019_1101_1125,0.153890
4,2019_1101_1133,0.675128


In [40]:
second_rd_submission.tail()

,ID,pred
2273,2019_1449_1459,0.264454
2274,2019_1449_1463,0.674249
2275,2019_1458_1459,0.332634
2276,2019_1458_1463,0.896041
2277,2019_1459_1463,0.881246


In [41]:
#Export to submit to Kaggle
second_rd_submission.to_csv('Data/~Created data/second_rd_submission.csv', index=False)

# Other models

No model performed as well as the neural network

XGBoost